In [1]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb
from quantfreedom.strategies.strategy_example import StrategyExample

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 04 seconds to download 2000 candles


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    long_or_short=np.array([LongOrShortType.Long]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    risk_account_pct_size=np.array([1]),
    max_equity_risk_pct=np.array([3,10]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]),
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]),
    trail_sl_by_pct=np.array([1.0]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    risk_reward=np.array([1,5]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    static_leverage=np.array([0.0]),
    num_candles=np.array([0]),
    entry_size_asset=np.array([0.0]),
    max_trades=np.array([3]),
    
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [11]:
strategy = StrategyExample(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    rsi_length=[14, 30],
    rsi_is_below=[50,80],
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 4
Total combinations of settings to test: 16

Total candles: 2,000
Total candles to test: 32,000


In [12]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
3,0,0,183.0,-92.03,11.48,-0.0000,-920.3387,79.6613
2,0,1,163.0,-91.44,8.59,-0.0000,-914.4162,85.5838
5,0,2,97.0,-95.06,9.28,-0.0000,-950.5565,49.4435
4,0,3,89.0,-95.04,3.37,-0.0000,-950.3506,49.6494
1,1,0,183.0,-91.15,12.02,-0.0000,-911.4521,88.5479
13,1,1,164.0,-91.57,7.93,-0.9145,-915.6777,84.3223
10,1,2,103.0,-95.35,8.74,-0.0000,-953.5209,46.4791
6,1,3,91.0,-95.06,3.30,-0.0000,-950.5891,49.4109
12,2,0,172.0,-89.90,11.63,-0.9078,-898.9659,101.0341
15,2,1,152.0,-90.15,7.89,-0.9304,-901.4709,98.5291


In [6]:
strategy.indicator_cart_product

IndicatorSettingsArrays(rsi_length=array([14, 14, 30, 30]), rsi_is_below=array([50, 80, 50, 80]))